In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
from numpy import load
!pip install transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.5 MB/s eta 0:00:00


In [3]:
folder = '/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/ILDC_expert/source'
file_names = os.listdir(folder)
print(file_names)

['1960_12.txt', '1953_14.txt', '1952_60.txt', '1951_64.txt', '1962_384.txt', '1961_344.txt', '1999_1001.txt', '1960_44.txt', '1962_113.txt', '1959_66.txt', '1951_10.txt', '1962_339.txt', '1962_118.txt', '1953_74.txt', '1961_365.txt', '1960_103.txt', '1960_327.txt', '1960_10.txt', '1963_37.txt', '2013_30.txt', '1951_80.txt', '1960_100.txt', '1960_265.txt', '1951_35.txt', '1954_114.txt', '1959_134.txt', '1962_128.txt', '1961_363.txt', '1960_72.txt', '1962_47.txt', '2013_57.txt', '1961_400.txt', '1954_144.txt', '1951_30.txt', '1959_76.txt', '1954_158.txt', '1954_13.txt', '1951_40.txt', '1951_33.txt', '1961_417.txt', '1959_5.txt', '2013_95.txt', '1960_87.txt', '1962_105.txt', '1962_207.txt', '2013_35.txt', '1954_0.txt', '1953_57.txt', '1952_42.txt', '1951_36.txt', '1959_26.txt', '1962_213.txt', '1953_26.txt', '2003_794.txt', '1952_75.txt', '1959_189.txt']


In [4]:
dataset = pd.read_csv('/content/drive/MyDrive/Thesis/train_data/ILDC_multi.csv') # loading dataset

# loading the corresponding label for each case in dataset
test = dataset.loc[dataset['split'] == 'test']

y_test0 = []
for i in range(test.shape[0]):
    y_test0.append(test.loc[i+33299,'label'])

In [5]:
test = test.reset_index(drop=True)
filter = test['name'].isin(file_names)
df = test[filter]
indexes = df.index

In [6]:
df

,text,label,split,name
51,criminal appellate jurisdiction criminal appea...,1,test,1962_213.txt
84,civil appellate jurisdiction review petition n...,0,test,1962_47.txt
123,civil appellate jurisdiction civil appeal numb...,0,test,1951_35.txt
127,\nmahajan j. \n\nthis is appeal from the judgm...,1,test,1953_74.txt
158,civil appellate jurisdiction civil appeal numb...,0,test,1960_100.txt
164,civil appellate jurisdiction civil appeal numb...,0,test,1960_72.txt
251,civil appellate jurisdiction civil appeal numb...,1,test,1963_37.txt
260,\nfazl ali j. \n\nthe principal question for d...,0,test,1951_33.txt
265,\nmahajan j. \n\npetition number 166 of 1951. ...,0,test,1952_42.txt
292,civil appellate jurisdiction civil appeal numb...,1,test,1953_26.txt


In [7]:
indexes

Int64Index([  51,   84,  123,  127,  158,  164,  251,  260,  265,  292,  315,
             369,  387,  594,  668,  701,  708,  714,  752,  791,  841,  850,
             870,  885,  894,  903,  936,  956,  987,  993, 1089, 1098, 1176,
            1180, 1191, 1206, 1251, 1294, 1299, 1319, 1327, 1359, 1387, 1415,
            1417, 1428, 1436, 1442, 1451, 1452, 1500, 1501, 1502, 1503, 1504,
            1506],
           dtype='int64')

In [9]:
path_transformer_chunk_embeddings_test = '/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/LSGBERT_bigru_att_full_old/LSGBERT_npy_files_cls_multi_full/LSGBERT_cls_test.npy'
x_test0 = load(path_transformer_chunk_embeddings_test, allow_pickle= True)

In [10]:
l=[]
for i in range(len(x_test0)):
  l.append(len(x_test0[i]))

In [11]:
print(l)

[7, 5, 9, 11, 12, 13, 11, 8, 25, 17, 25, 3, 4, 8, 11, 6, 10, 16, 14, 15, 13, 9, 4, 22, 25, 7, 13, 25, 14, 13, 25, 17, 12, 3, 10, 13, 9, 11, 14, 5, 15, 18, 9, 13, 15, 17, 7, 4, 8, 12, 7, 5, 17, 5, 25, 5, 14, 10, 8, 11, 16, 7, 8, 15, 18, 15, 25, 25, 14, 1, 25, 16, 17, 5, 4, 25, 5, 7, 14, 11, 9, 2, 15, 14, 7, 9, 9, 18, 23, 9, 24, 17, 6, 6, 5, 8, 13, 6, 4, 8, 13, 12, 13, 16, 20, 24, 12, 12, 25, 15, 25, 5, 16, 6, 6, 21, 10, 25, 14, 18, 16, 7, 10, 22, 22, 5, 21, 9, 7, 13, 3, 8, 19, 13, 7, 11, 20, 10, 10, 4, 19, 8, 12, 25, 11, 25, 14, 12, 20, 6, 9, 9, 8, 25, 2, 3, 7, 6, 6, 3, 14, 12, 25, 4, 6, 9, 4, 4, 25, 13, 3, 12, 10, 9, 5, 20, 9, 6, 7, 7, 9, 6, 9, 8, 6, 13, 7, 9, 18, 19, 6, 9, 12, 6, 22, 7, 16, 18, 7, 6, 9, 8, 9, 25, 10, 25, 11, 12, 3, 6, 13, 7, 25, 5, 11, 9, 11, 9, 6, 4, 2, 4, 25, 3, 6, 2, 13, 8, 8, 7, 6, 12, 25, 9, 5, 11, 10, 7, 21, 9, 20, 4, 19, 12, 5, 3, 7, 25, 7, 4, 6, 9, 9, 9, 25, 8, 7, 23, 7, 5, 10, 11, 3, 16, 9, 25, 9, 11, 8, 8, 10, 14, 11, 13, 12, 25, 15, 5, 25, 7, 14, 25, 4, 11,

In [12]:
def extract_and_count(l, index):
    element = l[index]
    count = l[:index].count(element)
    return element, count

In [13]:
elements = []
counts= []
for i in indexes:
  element, count = extract_and_count(l, i)
  elements.append(element)
  counts.append(count)

In [14]:
print(elements)

[5, 7, 22, 9, 6, 6, 9, 10, 25, 25, 8, 10, 5, 8, 6, 10, 6, 15, 11, 7, 7, 6, 6, 9, 4, 4, 7, 15, 6, 5, 3, 2, 9, 9, 9, 14, 16, 11, 4, 14, 7, 7, 5, 5, 3, 7, 16, 13, 4, 4, 6, 3, 1, 4, 19, 6]


In [15]:
print(counts)

[2, 6, 1, 8, 8, 9, 24, 10, 25, 30, 22, 17, 26, 41, 47, 47, 52, 15, 46, 60, 63, 64, 68, 69, 43, 44, 73, 22, 81, 85, 46, 22, 95, 96, 98, 36, 33, 69, 63, 39, 102, 106, 117, 120, 63, 114, 38, 65, 73, 74, 117, 69, 18, 78, 25, 118]


In [17]:
import pickle
file_path = "/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/LSGBERT_bigru_att_full_old/att_wt_of_LSGBERT_bigru_full.pkl"  # Puoi sostituire "my_array.pkl" con il percorso desiderato
with open(file_path, 'rb') as file:
    wt_vector = pickle.load(file)

In [18]:
output_dir = '/content/drive/MyDrive/Thesis/calibration/saved_model_multi_2560'
tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [19]:
# check if the dimensions of test embedding, occlusion scores and number of chunks for each documents is the same
att_scores_anno = []
x_test0_anno = []
for i in range(len(df)):
  index = df.index[i]
  all_toks = tokenizer.tokenize(df.iloc[i]['text'])
  if(len(all_toks) > 10000):
      all_toks = all_toks[len(all_toks)-10000:]
  splitted_toks = []
  total_tokens = len(all_toks)
  chunk_size = 510
  overlap_size = 100

  end = total_tokens
  if (end<101):
    splitted_toks.insert(0, all_toks)
  else:
    while end > 100:
      start = max(0, end - chunk_size)
      chunk = all_toks[start:end]
      splitted_toks.insert(0, chunk)
      if start!=0:
        end = start + overlap_size
      else:
        end=0

  CLS = tokenizer.cls_token
  SEP = tokenizer.sep_token
  e_sents = []
  for l_t in splitted_toks:
    l_t = [CLS] + l_t + [SEP]
    encoded_sent = tokenizer.convert_tokens_to_ids(l_t)
    e_sents.append(encoded_sent)

  e_sents = pad_sequences(e_sents, maxlen=512, value=0, dtype="long", padding="post")

  if len(e_sents) != len(x_test0[index]) or len(e_sents) != elements[i] or elements[i] != len(x_test0[index]):
    print(f'Dimension problem at index: {index}')
  else:
    att_scores_anno.append(wt_vector[(elements[i]-1)][counts[i]])
    x_test0_anno.append(x_test0[index])


Token indices sequence length is longer than the specified maximum sequence length for this model (8719 > 4096). Running this sequence through the model will result in indexing errors


In [20]:
att_anno = []

for item in att_scores_anno:
    att_anno.append(item[0])

In [21]:
# save the att scores of the annotated docs
#with open('/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/LSGBERT_bigru_att_full/att_weights_anno_docs_full.pkl', 'wb') as file:
#    pickle.dump(att_anno, file)

In [22]:
# extract the index of the most representative chunk in each doc
max_indexes = []
for i in range(len(att_anno)):
  index_max= att_anno[i].argmax()
  max_indexes.append(index_max)

In [23]:
print(max_indexes)

[4, 6, 21, 8, 5, 5, 8, 9, 24, 24, 7, 9, 4, 7, 5, 9, 5, 14, 10, 6, 6, 5, 5, 8, 3, 3, 6, 14, 5, 4, 2, 1, 8, 8, 8, 13, 15, 10, 3, 13, 6, 6, 4, 4, 2, 6, 15, 12, 3, 3, 5, 2, 0, 3, 18, 5]


In [24]:
for i in range(len(max_indexes)):
  if (max_indexes[i]+1 != len(att_anno[i])):
    print(f' NOT LAST AT INDEX {i}, chunk size: {len(att_anno[i])}, chunk chosen: {max_indexes[i]}')

In [25]:
def lsgbert_detok(tokens):
    # Initialize an empty list for the detokenized sequence
    detokenized_sequence = []

    # Initialize a variable for handling '##' signs
    current_token = ""

    # Iterate through the tokens in the sequence
    for token in tokens:
        if token.startswith("##"):
            # If the token starts with '##', remove '##' and append the rest of the token to the previous token
            current_token += token[2:]
        else:
            # If the token does not start with '##', check if it is a punctuation mark
            if current_token and token in string.punctuation:
                # If the current token is a punctuation mark, attach it to the previous token
                current_token += token
            else:
                # If the current token is not a punctuation mark, add the previous token (if present) to the detokenized sequence
                if current_token:
                    detokenized_sequence.append(current_token)
                # Add the current token to the detokenized sequence
                current_token = token

    # Make sure to add the last token to the detokenized sequence
    if current_token:
        detokenized_sequence.append(current_token)


    tokens_concat = []

    prev_tok=""

    for token in detokenized_sequence:
      if token.endswith('-'):
          prev_tok = token
      else:
          if (prev_tok != ""):
              token = prev_tok + token
              prev_tok = ""
          tokens_concat.append(token)

    if (prev_tok!=""):
        tokens_concat.append(prev_tok)

    # Join the tokens in the detokenized sequence into a string
    detokenized_text = " ".join(tokens_concat)

    return detokenized_text


In [26]:
final_explanations_att = {}
count_len = [ ]
for i in range(len(df)):
  index = df.index[i]
  all_toks = tokenizer.tokenize(df.iloc[i]['text'])
  if(len(all_toks) > 10000):
      all_toks = all_toks[len(all_toks)-10000:]
  splitted_toks = []
  total_tokens = len(all_toks)
  chunk_size = 510
  overlap_size = 100

  end = total_tokens
  if (end<101):
    splitted_toks.insert(0, all_toks)
  else:
    while end > 100:
      start = max(0, end - chunk_size)
      chunk = all_toks[start:end]
      splitted_toks.insert(0, chunk)
      if start!=0:
        end = start + overlap_size
      else:
        end=0


  explanation_of_this_doc = lsgbert_detok(splitted_toks[max_indexes[i]])
  doc_name = df.iloc[i]['name']
  final_explanations_att[doc_name] = explanation_of_this_doc


In [29]:
final_explanations_att

{'1962_213.txt': 'than once except in very exceptional circumstances. in this case the complainant had the fullest opportunity of adducing all the evidence that he was advised would be necessary to prove the charge against the accused person. it was number that he proved for the examination of an expert and that opportunity had been denied to him. the prosecution took its chance of having a decision in its favour on the evidence adduced by it before the trial companyrt. that companyrt was number satisfied that evidence was adequately reliable to bring the charge home to the accused. the accused was thus acquitted. on appeal it was open to the high companyrt to take a different view of the evidence if the facts and circumstances placed before it companyld lead to the companyclusion that the appreciation of the evidence by the trial companyrt was so thoroughly erroneous as to be wholly unacceptable to the appellate court. if the high companyrt companyld companye to the companyclusion it 

In [33]:
import json
json_file = open("/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/LSGBERT_bigru_att_full_old/att_exp_anno_full.json", "w")
json.dump(final_explanations_att, json_file)